In [1]:
!pip install -q grobid-client langchain openai faiss-cpu PyPDF2 tiktoken google-search-results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 16.3 MB/

In [2]:
import os
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationKGMemory
import requests
import re
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen


os.environ["OPENAI_API_KEY"] =  ""

In [3]:
def create_keywords(user_input):
    llm = OpenAI(temperature=0.9)

    template = """You are a helpful agent that generates keywords for a trailer story that match the user's input. The user's input is organized based on the '/' sign, first the genre of the story, second the mood of the story, and finally the plot of the story.First you need to construct five scenes from the user's input in the context of the plot. Then you need to generate representative keywords for the five scenes, using the genre and mood as a guide. Don't stray too far from the plot, and don't make it up.
    Example 1)
    user input: 로맨스 / 창백한, 잘생긴, 19금, 20대 / '결혼 조건. 서효주한테는 까맣게 잊을 만큼 쉬운 거였나?' '알아요. 제 역할이 뭐였는지. 그래서 쥐 죽은 듯이 살았다고요. 선배가 원하는 대로 온갖 눈치는 다 보면서, 있는 듯 없는 듯 그렇게 지냈어요. 자그마치 5년씩이나.' 그녀의 첫사랑은 실패였다. 사랑을 갈구했더니 혼인 계약서를 받았고, 시어머니 손에 아이를 유산 당했다. 이혼 후에는 두 번 다시 마주치는 일 따위 없을 줄 알았는데, 낡은 옥탑방으로 전남편이 찾아왔다. '재결합하자.'  '……그럴 마음 없어요.' '제안 아니야. 협박이지.' 얼마나 더 아파야 이 사랑을 끝낼 수 있을까. 얼마나 더 당해야 이 남자의 품에서 벗어날 수 있을까. 효주의 전남편이자 S그룹의 후계자 이명하는 또다시 그녀를 속박하려 했다.
    keywords: (장면1) 20대 여자, 창백한 얼굴, 예쁜\n(장면2) 집안, 비밀, 가문\n(장면3) 대표님, 남자, 잘생긴\n(장면4) 절망하는 남자, 유혹하는 여자, 19금\n(장면5) 꽃, 여자

    Example 2)
    user input: 판타지 / 잘생긴, 20대, 멋진 / 어느 날, 이세계로 소환되었다. 짐승들이 쏟아지고, 믿을 수 없는 위기가 닥쳐오나. 가지고 있는 재능은 밑바닥. '플레이어의 재능 수치는 최하입니다.' '거의 모든 수치가 절망적입니다.' 선택받은 용사든, 재능 있는 마법사들. 시간을 역행한 회귀자든. 모든 것을 이용해야 한다. 살아남기 위해. '쓰레기면 뭐 어떻습니까. 살아남기 위해서 뭔 짓인들 못 하겠어요?'
    keywords: (장면1) 이상세계, 으스스\n(장면2) 짐승, 위기\n(장면3) 용사, 마법사, 회귀자, 생존\n(장면4) 잘생긴, 남자, 20대\n(장면5) 잘생긴, 남자, 20대, 모험, 생존

    Example 3)
    user input: 로맨스 판타지 / 20대, 예쁜, 잘생긴 / '멍청한 여자, 죽어 버렸으면.' 남편 클라우스 공작은 악의 어린 방관자였고, '별볼일없는 친정이니 자식 교육이 이 모양이지.' 시어머니는 그녀를 머리부터 발끝까지 미워했으며, '네 남편이 사랑하는 사람은…… 나야.' 가장 가까운 친구는 그녀를 배신했다. 처참히 학대받는 공작 부인의 몸에서, 희대의 악녀 ‘천영령’이 깨어났다. 복수의 끝은 허망할 뿐이라고? 그건 진짜 복수를 해 본 적 없는 자들의 자기위로일 뿐이다. 원수의 무덤 위에서 벌이는 연회는 언제나 즐겁다.
    keywords: (장면1) 남편, 부정적, 20대, 악한\n(장면2) 시어머니, 며느리, 미움\n(장면3) 친구, 배신, 불륜\n(장면4) 학대, 여자, 20대, 슬픈\n(장면5) 악녀, 복수, 원수

    Relavant Information:
    {history}

    Conversation:
    user input: {input}
    keywords:"""

    prompt = PromptTemplate(
        input_variables=["history", "input"],
        template=template
    )

    memory = ConversationKGMemory(llm=llm)
    memory.save_context({"input":"'결혼 조건. 서효주한테는 까맣게 잊을 만큼 쉬운 거였나?' '알아요. 제 역할이 뭐였는지. 그래서 쥐 죽은 듯이 살았다고요. 선배가 원하는 대로 온갖 눈치는 다 보면서, 있는 듯 없는 듯 그렇게 지냈어요. 자그마치 5년씩이나.' 그녀의 첫사랑은 실패였다. 사랑을 갈구했더니 혼인 계약서를 받았고, 시어머니 손에 아이를 유산 당했다. 이혼 후에는 두 번 다시 마주치는 일 따위 없을 줄 알았는데, 낡은 옥탑방으로 전남편이 찾아왔다. '재결합하자.'  '……그럴 마음 없어요.' '제안 아니야. 협박이지.' 얼마나 더 아파야 이 사랑을 끝낼 수 있을까. 얼마나 더 당해야 이 남자의 품에서 벗어날 수 있을까. 효주의 전남편이자 S그룹의 후계자 이명하는 또다시 그녀를 속박하려 했다."}, {"output":"(장면1) 그녀의 첫사랑은 실패했다.\n(장면2) 사랑을 갈구했더니 혼인 계약서를 받았고, 시어머니 손에 아이를 유산 당했다. \n(장면3) 이혼 후 낡은 옥탑방에 사는 그녀를 찾아온 전남편 이명하. \n(장면4) 아프게만 했던 전남편의 첫마디. '재결합하자'\n(장면5) 효주의 전남편이자 S그룹의 후계자 이명하는 또다시 그녀를 속박하려 하는데 둘의 숨막히는 애증 관계가 궁금하시다면, 웹 소설 <우아한 재혼>에서 확인하실 수 있습니다."})

    conversation_with_kg = ConversationChain(
        llm=llm,
        verbose=True,
        prompt=prompt,
        memory=memory
    )

    keywords = conversation_with_kg.predict(input=user_input)

    return keywords


In [4]:
create_keywords("로맨스 / 20대, 예쁜, 잘생긴 / '좀 봐주라, 응? 산 사람은 살아야지. 넌 어차피 죽을 거잖아, 흑…….' 시한부를 선고받은 내 앞에서, 하나뿐인 친구가 애처롭게 울었다.  나는 그 시한부조차 채우지 못하고 남편의 손에 죽었다. 그리고. '지원 씨, 점심시간 다 끝났어!' 10년 전, 내가 다니던 회사에서 깨어났다. 죽음 이후에 주어진 또 한 번의 삶.  운명을 바꾸려면 내 시궁창 같은 운명에 누군가 대신 들어가야 한다. 나는 그 '누군가'를 내 친구로 정했다. 너, 내 남편 원했잖아.")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful agent that generates keywords for a trailer story that match the user's input. The user's input is organized based on the '/' sign, first the genre of the story, second the mood of the story, and finally the plot of the story.First you need to construct five scenes from the user's input in the context of the plot. Then you need to generate representative keywords for the five scenes, using the genre and mood as a guide. Don't stray too far from the plot, and don't make it up.
    Example 1)
    user input: 로맨스 / 창백한, 잘생긴, 19금, 20대 / '결혼 조건. 서효주한테는 까맣게 잊을 만큼 쉬운 거였나?' '알아요. 제 역할이 뭐였는지. 그래서 쥐 죽은 듯이 살았다고요. 선배가 원하는 대로 온갖 눈치는 다 보면서, 있는 듯 없는 듯 그렇게 지냈어요. 자그마치 5년씩이나.' 그녀의 첫사랑은 실패였다. 사랑을 갈구했더니 혼인 계약서를 받았고, 시어머니 손에 아이를 유산 당했다. 이혼 후에는 두 번 다시 마주치는 일 따위 없을 줄 알았는데, 낡은 옥탑방으로 전남편이 찾아왔다. '재결합하자.'  '……그럴 마음 없어요.' '제안 아니야. 협박이지.' 얼마나 더 아파야 이 사랑을 끝낼 수 있을까. 얼마나 더 당해야 이 남자의 품에서 벗어날 수 있을까. 효주의 전남편이자 S그룹의 후계자 이명하는 또다시 그녀를 속박

' (장면1) 우울, 죽음, 애처로운, 친구\n(장면2) 시한부, 죽음, 남편\n(장면3) 회사, 깨어남, 운명, 친구\n(장면4) 삶, 운명, 선택, 친구\n(장면5) 삶, 선택, 질투, 남편'